In [8]:
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [7]:
s3_private_path_tsv = "s3://{}/homework-2/csv".format(bucket)
print(s3_private_path_tsv)

s3://sagemaker-us-east-1-399018723364/homework-2/csv


In [8]:
%store s3_private_path_tsv

Stored 's3_private_path_tsv' (str)


In [13]:
!aws s3 cp /home/sagemaker-user/aai-540-homework/homework-2-1/data/dataset.csv $s3_private_path_tsv/

upload: ../../../aai-540-homework/homework-2-1/data/dataset.csv to s3://sagemaker-us-east-1-399018723364/homework-2/csv/dataset.csv


In [9]:
print(s3_private_path_tsv)

s3://sagemaker-us-east-1-399018723364/homework-2/csv


In [10]:
!aws s3 ls $s3_private_path_tsv/

2025-01-16 02:52:15   20118244 dataset.csv


In [12]:
import boto3
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

In [13]:
ingest_create_athena_db_passed = False

In [15]:
%store -r s3_private_path_tsv

In [16]:
try:
    s3_private_path_tsv
except NameError:
    print("*****************************************************************************")
    print("[ERROR] PLEASE RE-RUN THE PREVIOUS COPY TSV TO S3 NOTEBOOK ******************")
    print("[ERROR] THIS NOTEBOOK WILL NOT RUN PROPERLY. ********************************")
    print("*****************************************************************************")

In [17]:
from pyathena import connect

In [18]:
database_name = "dsoaws"

In [19]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [20]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [21]:
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
print(statement)

CREATE DATABASE IF NOT EXISTS dsoaws


In [22]:
import pandas as pd

pd.read_sql(statement, conn)

/tmp/ipykernel_1065/3803073958.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


## Verfiy Database Creation

In [23]:
statement = "SHOW DATABASES"

df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_1065/3999478089.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,database_name
0,default
1,dsoaws


In [24]:
if database_name in df_show.values:
    ingest_create_athena_db_passed = True

In [25]:
%store ingest_create_athena_db_passed

Stored 'ingest_create_athena_db_passed' (bool)


In [26]:
%store

Stored variables and their in-db values:
ingest_create_athena_db_passed             -> True
s3_private_path_tsv                        -> 's3://sagemaker-us-east-1-399018723364/homework-2/
setup_dependencies_passed                  -> True
setup_s3_bucket_passed                     -> True


## Release Resources

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}